## Benchmarking Classifier

In [1]:
from tabulate import tabulate
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from gensim.models.word2vec import Word2Vec
from collections import Counter, defaultdict
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import StratifiedShuffleSplit
import os
import re
#import magic
import collections
from tqdm import tqdm


# GLOVE_42B_300D_PATH = "/media/rishikesh/E6488E95488E6461/Projects/Word2Vec/glove.42B.300d.txt"

C:\Anaconda3\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
doc=[]

for root, directories, filenames in os.walk("./test/"):
     for file in filenames:
            if file != ".DS_Store":
                file_name, file_extension = os.path.splitext(file)
                # filename.append("{}/{}".format(root,file))
                # filetype.append(file_extension)
                # magic_type.append(magic.from_file("{}/{}".format(root,file), mime=True))
                file1 = open("{}/{}".format(root,file),"r",encoding='utf-8', errors='ignore')
                doc.append(file1.read())

df=pd.DataFrame()
df['doc']=doc

In [3]:
doc=[]
# filename=[]
classes=[]
# filetype=[]
# magic_type=[]

In [4]:
for root, directories, filenames in os.walk('./data/'):
     for file in filenames:
            if file != ".DS_Store":
                file_name, file_extension = os.path.splitext(file)
                # filename.append("{}/{}".format(root,file))
                classes.append(root.split("/")[-1])
                # filetype.append(file_extension) 
                # magic_type.append(magic.from_file("{}/{}".format(root,file), mime=True))
                file1 = open("{}/{}".format(root,file),"r",encoding='utf-8', errors='ignore')
                doc.append(file1.read())

In [5]:
df=pd.DataFrame()
df['doc']=doc
df['classes']=classes
# df['file']=filename
# df['type']=filetype
# df['magic_type']=magic_type

In [6]:
df.head()

,doc,classes
0,\nSmiley Structural Engineering \n \n\n1430 Fl...,receipts
1,\nInvoice\n\nGoogle Inc.\n1600 Amphitheatre Pk...,receipts
2,\nInvoice\n\nGoogle Inc.\n1600 Amphitheatre Pk...,receipts
3,\nInvoice\n\nGoogle Inc.\n1600 Amphitheatre Pk...,receipts
4,\n650 PONCE DE LEON AVE 404-892-8042\nHOLLY_D_...,receipts


In [7]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
 
stop_words = set(stopwords.words('english'))
filtered_sentences=[]
for text in df['doc'].values:
        word_tokens = word_tokenize(text)
        filtered_sentence = [w for w in word_tokens if not w in stop_words]
        filtered_sentences.append(filtered_sentence)

In [8]:
df['filtered_doc']=filtered_sentences
df.head()

,doc,classes,filtered_doc
0,\nSmiley Structural Engineering \n \n\n1430 Fl...,receipts,"[Smiley, Structural, Engineering, 1430, Flatta..."
1,\nInvoice\n\nGoogle Inc.\n1600 Amphitheatre Pk...,receipts,"[Invoice, Google, Inc., 1600, Amphitheatre, Pk..."
2,\nInvoice\n\nGoogle Inc.\n1600 Amphitheatre Pk...,receipts,"[Invoice, Google, Inc., 1600, Amphitheatre, Pk..."
3,\nInvoice\n\nGoogle Inc.\n1600 Amphitheatre Pk...,receipts,"[Invoice, Google, Inc., 1600, Amphitheatre, Pk..."
4,\n650 PONCE DE LEON AVE 404-892-8042\nHOLLY_D_...,receipts,"[650, PONCE, DE, LEON, AVE, 404-892-8042, HOLL..."


In [9]:
X, y = [], []
# with open(TRAIN_SET_PATH, "rb") as infile:
#    for line in infile:
#        label, text = line.split("\t")
        # texts are already tokenized, just split on space
        # in a real case we would use e.g. spaCy for tokenization
        # and maybe remove stopwords etc.
#        X.append(text.split())
#        y.append(label)
y=pd.get_dummies(df['classes'])
X, y = np.array(df['filtered_doc'].values), np.array(y)
print ("total examples %s" % len(y))

total examples 63


In [10]:
len(all_words)

NameError: name 'all_words' is not defined

In [12]:
all_words = set(w for words in X for w in words)

In [13]:
glove_big = {}
with open(GLOVE_42B_300D_PATH, "rb") as infile:
    for line in infile:
        parts = line.split()
        word = parts[0]
        nums = map(float, parts[1:])
        if word in all_words:
            glove_big[word] = np.array(nums)         

In [11]:
# train word2vec on all the texts - both training and test set
# we're not using test labels, just texts so this is fine
model = Word2Vec(X, size=100, window=5, min_count=5, workers=2)
w2v = {w: vec for w, vec in zip(model.wv.index2word, model.wv.syn0)}

In [10]:
# start with the classics - naive bayes of the multinomial and bernoulli varieties
# with either pure counts or tfidf features
mult_nb = Pipeline([("count_vectorizer", CountVectorizer(analyzer=lambda x: x)), ("multinomial nb", MultinomialNB())])
bern_nb = Pipeline([("count_vectorizer", CountVectorizer(analyzer=lambda x: x)), ("bernoulli nb", BernoulliNB())])
mult_nb_tfidf = Pipeline([("tfidf_vectorizer", TfidfVectorizer(analyzer=lambda x: x)), ("multinomial nb", MultinomialNB())])
bern_nb_tfidf = Pipeline([("tfidf_vectorizer", TfidfVectorizer(analyzer=lambda x: x)), ("bernoulli nb", BernoulliNB())])
# SVM - which is supposed to be more or less state of the art 
# http://www.cs.cornell.edu/people/tj/publications/joachims_98a.pdf
# svc = Pipeline([("count_vectorizer", CountVectorizer(analyzer=lambda x: x)), ("linear svc", SVC(kernel="linear"))])
# svc_tfidf = Pipeline([("tfidf_vectorizer", TfidfVectorizer(analyzer=lambda x: x)), ("linear svc", SVC(kernel="linear"))])

In [11]:
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = len(word2vec.items())
    
    def fit(self, X, y):
        return self 

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec] 
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])
    
# and a tf-idf version of the same

class TfidfEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.word2weight = None
        self.dim = len(word2vec.items())
        
    def fit(self, X, y):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        # if a word was never seen - it must be at least as infrequent
        # as any of the known words - so the default idf is the max of 
        # known idf's
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf, 
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])
        
        return self
    
    def transform(self, X):
        return np.array([
                np.mean([self.word2vec[w] * self.word2weight[w]
                         for w in words if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ])

In [14]:
y=pd.get_dummies(df['classes'])

In [17]:
y.shape[1]

3

In [ ]:
colname = df.columns[pos]

In [22]:
label_dict={}

In [24]:

for i in range(y.shape[1]):
    label_dict[i]=y.columns[i]

print(label_dict)   

{0: 'receipts', 1: 'taxes', 2: 'tickets'}


In [15]:
y=

,receipts,taxes,tickets
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0
5,1,0,0
6,1,0,0
7,1,0,0
8,1,0,0
9,1,0,0


In [28]:
# Extra Trees classifier is almost universally great, let's stack it with our embeddings
# etree_glove_small = Pipeline([("glove vectorizer", MeanEmbeddingVectorizer(glove_small)), 
#                        ("extra trees", ExtraTreesClassifier(n_estimators=200))])
#etree_glove_small_tfidf = Pipeline([("glove vectorizer", TfidfEmbeddingVectorizer(glove_small)), 
#                        ("extra trees", ExtraTreesClassifier(n_estimators=200))])


etree_w2v = Pipeline([("word2vec vectorizer", MeanEmbeddingVectorizer(w2v)), 
                        ("extra trees", ExtraTreesClassifier(n_estimators=200))])
etree_w2v_tfidf = Pipeline([("word2vec vectorizer", TfidfEmbeddingVectorizer(w2v)), 
                        ("extra trees", ExtraTreesClassifier(n_estimators=200))])




all_models = [
    ("mult_nb", mult_nb),
    ("mult_nb_tfidf", mult_nb_tfidf),
    ("bern_nb", bern_nb),
    ("bern_nb_tfidf", bern_nb_tfidf),
    #("svc", svc),
    #("svc_tfidf", svc_tfidf),
    #("glove_small", etree_glove_small), 
    #("glove_small_tfidf", etree_glove_small_tfidf),
    #("glove_big", etree_glove_big), 
    #("glove_big_tfidf", etree_glove_big_tfidf),
    
]


In [20]:
mult_nb_tfidf.fit(X,y)

Pipeline(steps=[('tfidf_vectorizer', TfidfVectorizer(analyzer=<function <lambda> at 0x000001DE45A0F268>,
        binary=False, decode_error='strict', dtype=<class 'numpy.int64'>,
        encoding='utf-8', input='content', lowercase=True, max_df=1.0,
        max_features=None, min_df=1, ngram_range=(1, 1), no...  vocabulary=None)), ('multinomial nb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [12]:
y = np.argmax(y, axis=1)

In [13]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2], dtype=int64)

In [26]:
cross_val_score(etree_w2v, X, y, cv=5).mean()

ValueError: setting an array element with a sequence.

In [29]:
scores=[]
for name, model in all_models :
    scores.append([name,cross_val_score(model, X, y, cv=5).mean()])

In [30]:
scores = sorted([(name, cross_val_score(model, X, y, cv=5).mean())
                 for name, model in all_models])

In [31]:
print(tabulate(scores, floatfmt=".4f", headers=("model", 'score')))

model            score
-------------  -------
bern_nb         0.5538
bern_nb_tfidf   0.5538
mult_nb         0.9038
mult_nb_tfidf   0.8538


In [13]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.externals import joblib
import pickle as cPickle
import os
import re
import collections
from tqdm import tqdm
import json

In [14]:
# load the user configs
with open('./conf/config.json') as f:
	config = json.load(f)

# config variables
data_path = config["data_path"]
random_seed = config["seed"]
model = config["model"]
model_path = config["model_path"]
test_size = config["test_size"]

doc=[]
classes=[]
# filename=[]
# filetype=[]
# magic_type=[]
if not os.path.exists(model_path):
    os.mkdir(model_path)

for root, directories, filenames in os.walk(data_path):
     for file in filenames:
            if file != ".DS_Store":
                file_name, file_extension = os.path.splitext(file)
                classes.append(root.split("/")[-1])
                # filename.append("{}/{}".format(root,file))
                # filetype.append(file_extension)
                # magic_type.append(magic.from_file("{}/{}".format(root,file), mime=True))
                file1 = open("{}/{}".format(root,file),"r",encoding='utf-8', errors='ignore')
                doc.append(file1.read())

df=pd.DataFrame()
df['doc']=doc
df['classes']=classes
# df['file']=filename
# df['type']=filetype

stop_words = set(stopwords.words('english'))
filtered_sentences=[]
for text in df['doc'].values:
        word_tokens = word_tokenize(text)
        filtered_sentence = [w for w in word_tokens if not w in stop_words]
        filtered_sentences.append(filtered_sentence)

df['filtered_doc']=filtered_sentences

X, y = [], []

y=pd.get_dummies(df['classes'])
X, y = np.array(df['filtered_doc'].values), np.array(y)

array([ '\nSmiley Structural Engineering \n \n\n1430 Flattail Run \n\nLawrenceville, GA 30043 \n\nPhone 678-720-8189 \n\nINVOICE \n\nINVOICE #0233 \n\nDATE: OCTOBER 14, 2016 \n\n \n\nTO: \n\nPritesh Patel \n2924 Clairmont Rd \nAtlanta, GA 30329 \n\n \n\n \n\nFOR: \n\n6414 Thornhedge Dr \nProject#16150 \n\n \n\nDESCRIPTION AMOUNT \n\n  \n\nProvide site visit to document existing condition and provide structural assessment $400.00 \n\n  \n\n  \n\n  \n\n  \n\n  \n\n  \n\n  \n\n  \n\n  \n\n  \n\n  \n\n  \n\n  \n\n  \n\n  \n\nTOTAL   $400.00 \n\nMake all checks payable to Smiley Structural Engineering \n\nTotal due Upon Receipt. Overdue accounts subject to a service charge of 1% per month.  \n\n Thank you for your business! \n\n\n',
       '\nInvoice\n\nGoogle Inc.\n1600 Amphitheatre Pkwy\nMountain View, CA 94043\nUSA\nFederal Tax ID: 77-0493581\n\nBill to\nPritesh Patel\n803 Calibre Woods Dr.\nAtlanta, GA 30329\nUnited States\n\nDetails\nInvoice number: 1735542805655192-46\nIssue date: Aug

In [8]:
from nltk.corpus import wordnet as wn

In [9]:
class StemTokenizer(object):
    def __init__(self):
        self.ignore_set = {'footnote', 'nietzsche', 'plato', 'mr.'}

    def __call__(self, doc):
        words = []
        for word in word_tokenize(doc):
            word = word.lower()
            w = wn.morphy(word)
            if w and len(w) > 1 and w not in self.ignore_set:
                words.append(w)
        return words

In [42]:
from nltk.stem import SnowballStemmer
from nltk import word_tokenize
stemmer = SnowballStemmer('english').stem
def stem_tokenize(text):
    return [stemmer(i) for i in word_tokenize(text)]

vectorizer = CountVectorizer(analyzer='word',lowercase=True,tokenizer=stem_tokenize)
X_train = vectorizer.fit_transform(df.doc.values)
with open('vectorizer.pk', 'wb') as fin:
    cPickle.dump(vectorizer, fin)

In [ ]:
if model == 'mult_nb_tfidf':
    tfidf = TfidfVectorizer(analyzer=lambda x: x)
    # X = tfidf.fit_transform(X)
    vectorizer =CountVectorizer()
    X_train = vectorizer.fit_transform(X)
    joblib.dump(tfidf, "{}{}_Vect.pkl".format(model_path,model))
elif model == 'mult_nb':
    count = CountVectorizer(analyzer=lambda x: x)
    X = count.fit_transform(X)
    joblib.dump(count, "{}{}_Vect.pkl".format(model_path,model))

In [44]:


clf = MultinomialNB()
# mult_nb_tfidf = Pipeline([("tfidf_vectorizer", TfidfVectorizer(analyzer=lambda x: x)), ("multinomial nb", MultinomialNB())])
# mult_nb = Pipeline([("count_vectorizer", CountVectorizer(analyzer=lambda x: x)), ("multinomial nb", MultinomialNB())])

#y = np.argmax(y, axis=1)

# all_models = {"mult_nb":mult_nb,"mult_nb_tfidf":mult_nb_tfidf}

# clf=all_models.get(model)

clf.fit(X_train, y)

print(cross_val_score(MultinomialNB(), X_train, y, cv=5).mean())


0.803846153846


In [43]:
print(cross_val_score(MultinomialNB(), X_train, y, cv=5).mean())

0.803846153846


In [48]:
# load the user configs
with open('conf/config.json') as f:
	config = json.load(f)

# config variables
data_path = config["data_path"]
random_seed = config["seed"]
model = config["model"]
model_path = config["model_path"]
file_ = config["file"]
test = config["test_path"]
result = config["output_path"]

stemmer = SnowballStemmer('english').stem
def stem_tokenize(text):
     return [stemmer(i) for i in word_tokenize(text)]

# clf = joblib.load("{}{}_clf.pkl".format(model_path,model))
tfidf = joblib.load("{}{}_vectorizer.pkl".format(model_path,model))
doc=[]

for root, directories, filenames in os.walk(test):
     for file in filenames:
            if file != ".DS_Store":
                file_name, file_extension = os.path.splitext(file)
                # filename.append("{}/{}".format(root,file))
                # filetype.append(file_extension)
                # magic_type.append(magic.from_file("{}/{}".format(root,file), mime=True))
                file1 = open("{}/{}".format(root,file),"r",encoding='utf-8', errors='ignore')
                doc.append(file1.read())

df=pd.DataFrame()
df['doc']=doc



X = tfidf.transform(df.doc.values)

pred = clf.predict(X)

df["pred"]=pred

In [49]:
df.pred

0    2
1    2
2    2
3    0
4    0
5    0
6    0
7    0
Name: pred, dtype: int64

In [47]:
clf.classes_

array([0, 1, 2], dtype=int64)

In [ ]:
clf.predict()